## 데이터 불러오기 : 어제했던 08_와인데이터

In [4]:
### 08_wine.csv 파일 불러들이시고 출력까지
import pandas as pd

In [10]:
wine = pd.read_csv('./data/08_wine.csv', encoding = 'utf-8-sig')

In [20]:
wine

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0
...,...,...,...,...
6492,11.2,1.6,3.27,1.0
6493,9.6,8.0,3.15,1.0
6494,9.4,1.2,2.99,1.0
6495,12.8,1.1,3.34,1.0


In [13]:
### 독립변수와 종속변수 데이터 추출하기
# 독립변수명 : data
# 종속변수명 : target

data = wine[['alcohol','sugar','pH']].to_numpy()
target=wine['class'].to_numpy()

print(data.shape, target.shape)

(6497, 3) (6497,)


### 교차검증 : train_test_split() 방식(가장기본 : 엄밀히 말하면 교차는 아님)

In [14]:
### 훈련데이터 : 검증데이터 : 테스트데이터로 쪼개기
# - 보통 6 : 2 : 2로 쪼갭니다.
# - 보통 많이 사용하는 방식 ..

In [15]:
### 1. 훈련데이터와 테스트데이터를 8 : 2로 쪼개기
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = \
        train_test_split(data, target, test_size = 0.2, random_state = 42)

print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(5197, 3) (5197,)
(1300, 3) (1300,)


In [16]:
### 2. 훈련데이터를 이용해서 검증데이터 생성하기
### 1. 훈련데이터와 테스트데이터를 8 : 2로 쪼개기

sub_input, val_input, sub_target, val_target = \
        train_test_split(train_input, train_target, test_size = 0.2, random_state = 42)

print(sub_input.shape, sub_target.shape)
print(val_input.shape, val_target.shape)

(4157, 3) (4157,)
(1040, 3) (1040,)


In [23]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(sub_input, sub_target)
sub_scaled = ss.transform(sub_input)
val_scaled = ss.transform(val_input)
test_scaled = ss.transform(test_input)

In [24]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_scaled, sub_target)
print('sub결정력 = ' , dt.score(sub_scaled, sub_target))
print('val결정력 = ' , dt.score(val_scaled, val_target))

sub결정력 =  0.9971133028626413
val결정력 =  0.8625


##### 질문 하이퍼파라미터 ??

In [26]:
print(dt.score(test_scaled, test_target))

0.8561538461538462


In [27]:
## test데이터는 사람이 개입하는것을 싫어한다
## 테스트데이터를 기준으로 하이퍼파라미터를 최적화하므로
## 검증데이터로 훈련을 계속 시키고
## 마지막에 사용하지않은 test데이터를 한 번 넣어보는 것

## 교차검증 시작 ...

In [28]:
### 훈련데이터와 검증데이터를 구분할 필요없이
# - 내부적으로 구분해서 사용
# - 내부적으로 구분에 사용하는 데이터는 훈련(train) 데이터
# - 데이터 준비는 기존처럼 훈련과 테스트 데이터만 준비..
# - 검증데이터는 별도로 준비 안해도 됨...

In [31]:
### 교차검증에 사용되는 주요 키워드
# 3-폴드(fold) 교차검증
#  : 훈련데이터를 세부분으로 나눠서 데이터를 쪼개고 수행
#  : 구간마다 훈련데이터를 검증데이터로 바꿔가면서 수행
#  : k-폴드 교차검증 또는 k-겹 교차검증이라고도 합니다.

# 보통 5-폴드 교차검증 또는 10-폴드 교차검증을 사용합니다.
#  : 훈련데이터로 80% 이상을 모두 모델 훈련에 사용 가능

In [32]:
### 사용하는 데이터 다시한번 확인
print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(5197, 3) (5197,)
(1300, 3) (1300,)


In [43]:
### 교차검증 모듈 : cross_validate
from sklearn.model_selection import cross_validate
import numpy as np
# dt : 결정트리 훈련모델(다른 모델을 사용한 경우 해당 모델)
# 두번째값 : 훈련데이터 (fold에서 훈련데이터 쪼갤때 사용)
# 세번째값 : 검증데이터 (fold에서 검증데이터 쪼갤때 사용)
scores = cross_validate(dt, train_input, train_target)
print(scores)

### 딕셔너리의 test_score의 평균값이 최종 훈련모델의 평가점수(정확도)
print('최종 score = ', scores['test_score'].mean())
print('최종 score = ', np.mean(scores['test_score']))

{'fit_time': array([0.01286888, 0.01296592, 0.0135498 , 0.01334691, 0.01492   ]), 'score_time': array([0.00111103, 0.00199461, 0.00141048, 0.0020268 , 0.00199604]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
최종 score =  0.855300214703487
최종 score =  0.855300214703487


In [64]:
# fit_time : 훈련소요시간
# score_time : 스코어에대한 검증시간
# test_score : 각 fold마다의 평균최종값

In [57]:
### 훈련 데이터를 섞거나, 폴드의 갯수를 지정할 수 있는 클래스
# - Stratified<Fold : 
from sklearn.model_selection import StratifiedKFold

# 교차검증 함수 그대로 사용
# cv : 분할기 속성
#    : 분할기로 StratifiedKFold 클래스 사용
#    : 속성값이 없을 경우 기본 fold는 5, 기본 섞지는 않음
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
# print(scores)
print('최종 score = ', np.mean(scores['test_score']))


# 교차검증 함수 그대로 사용
# cv : 분할기 속성
#    : 분할기로 StratifiedKFold 클래스 사용
#    : 속성값이 없을 경우 기본 fold는 5, 기본 섞지는 않음


# n_splits : fold 갯수, shuffle : 섞기
splitter = StratifiedKFold(n_splits = 20, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)
print('최종 score = ', np.mean(scores['test_score']))


최종 score =  0.855300214703487
최종 score =  0.8603073953073954


## 하이퍼파라미터 튜닝(AutoML)
#### score 최적화


In [144]:
### 교차검증, 하이퍼파라미터 찾기, 모델훈련을 한번에 자동으로 수행
# - 이런 개념을 AutoML 이라고 합니다.
# - 사용패키지 : sklearn.model_selection
# - 사용클래스 : GridSearchCV
# - 그리드 서치라고도 합니다

## 결정트리의 max_depth(트리의 깊이) 값 찾기
##### - 그리드서치(GridSearchCV) 객체(모델) 사용

In [63]:
from sklearn.model_selection import GridSearchCV

# 찾을 하이퍼파라미터는 딕셔너리로
# 실제 파라미터 변수이름 그대로, 찾을 범위 지정
params = {'max_depth' : range(5, 20, 1)}

# 객체(모델) 생성
# - 첫번째 값 : 훈련모델 넣기
# - 두번째 값 : 찾을 하이퍼파라메터 값들(딕셔너리로 정의)
# - 세번째 값 : CPU 코어 갯수 (-1은 모든 코어 사용 병렬 처리)

gs = GridSearchCV(DecisionTreeClassifier(random_state=42),
                  params, n_jobs = -1)

# 훈련시키기
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20)})

In [65]:
## sklearn 공식문서
## 모델 여러개 쓸줄 알아야 하는 이유 ? 하나하나하나 증명을 해야한다, 비교하는 모델들을 알고있어야한다
## ensemble모델이 상당히 막강한데 그래도 비교대상이 있어야한다

In [68]:
# 그리드서치의 결과
# - 하이퍼파라미터의 찾은 값 알려줌
# - 중요 : 훈련모델이 생성되어 반한됨
# 최적의 하이퍼파라미터 값 출력
print(gs.best_estimator_)

DecisionTreeClassifier(max_depth=8, random_state=42)


In [74]:
### 그리드서치를 통해 생성된 훈련모델을 통해
# 평가해보기, 훈련데이터 전체 사용
dt= gs.best_estimator_
print(dt.score(train_input, train_target))

# 이렇게하는거 아니었어?
print(gs.score(train_input,train_target))

0.9003271117952665


0.9003271117952665

In [73]:
print(gs.best_params_)

{'max_depth': 8}


In [78]:
# gird serch가 찾은 모든 값
# 딕셔너리의 key값만 조회하기
print(gs.cv_results_.keys())

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])


In [79]:
### params 값 확인
print(gs.cv_results_['params'])

[{'max_depth': 5}, {'max_depth': 6}, {'max_depth': 7}, {'max_depth': 8}, {'max_depth': 9}, {'max_depth': 10}, {'max_depth': 11}, {'max_depth': 12}, {'max_depth': 13}, {'max_depth': 14}, {'max_depth': 15}, {'max_depth': 16}, {'max_depth': 17}, {'max_depth': 18}, {'max_depth': 19}]


In [104]:
gs.cv_results_['params'][1]['max_depth']

6

In [80]:
### 1폴드 조회하기
### 찾으라고 한 depth 개수만큼 쪼갰다
print(gs.cv_results_['split0_test_score'])

[0.84711538 0.84807692 0.85769231 0.85288462 0.85769231 0.84519231
 0.85865385 0.86730769 0.86538462 0.86826923 0.86730769 0.86442308
 0.86346154 0.86634615 0.87211538]


In [99]:
sec0 = np.column_stack([gs.cv_results_['split0_test_score'], gs.cv_results_['params']])
sec1 = np.column_stack([gs.cv_results_['split1_test_score'], gs.cv_results_['params']])
sec2 = np.column_stack([gs.cv_results_['split2_test_score'], gs.cv_results_['params']])
sec3 = np.column_stack([gs.cv_results_['split3_test_score'], gs.cv_results_['params']])
sec4 = np.column_stack([gs.cv_results_['split4_test_score'], gs.cv_results_['params']])

df0 = pd.DataFrame(data=sec0, columns = ['max_depth','score'])
df1 = pd.DataFrame(data=sec1, columns = ['max_depth','score'])
df2 = pd.DataFrame(data=sec2, columns = ['max_depth','score'])
df3 = pd.DataFrame(data=sec3, columns = ['max_depth','score'])
df4 = pd.DataFrame(data=sec4, columns = ['max_depth','score'])


df0

,max_depth,score
0,0.847115,{'max_depth': 5}
1,0.848077,{'max_depth': 6}
2,0.857692,{'max_depth': 7}
3,0.852885,{'max_depth': 8}
4,0.857692,{'max_depth': 9}
5,0.845192,{'max_depth': 10}
6,0.858654,{'max_depth': 11}
7,0.867308,{'max_depth': 12}
8,0.865385,{'max_depth': 13}
9,0.868269,{'max_depth': 14}


In [81]:
### 훈련모델 최종 검증하기
print(dt.score(test_input, test_target))

0.8584615384615385


## 찾을 하이퍼파라미터 여러개 사용하기

In [108]:
### 하이퍼 파라미터 여러개 지정
# min_inpurity_decrease : 결정트리 노드 분리시 불순도 감도 최소량
# max_depth : 결정트리 깊이
# min_samples_split : 결정트리 노드를 나누기 위한 최소 샘플 갯수
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001) ,
          'max_depth' : range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)}


### 그리드서치 훈련시키기
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),
                 params, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [110]:
# 파라미터 3개가 동시에 만족하는 최적값
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [114]:
### 그리드서치를 통해 생성된 훈련모델을 통해
# 평가해보기, 훈련데이터 전체 사용
dt= gs.best_estimator_
print('훈련data = ',dt.score(train_input, train_target))
print('최종data = ',dt.score(test_input,test_target))

## (해석)
# 0.03정도 차이나므로 굉장히 우수한 모델임

훈련data =  0.892053107562055
최종data =  0.8615384615384616


In [115]:
## 파라미터 범위를 serch하는 횟수에 따라 폴드개수가 결정되는건지
## 폴드개수를 정할수있는건지

In [118]:
### 10 - 폴드 지정하기
splitter = StratifiedKFold(n_splits = 7, shuffle = True, random_state = 42)

### 하이퍼 파라미터 여러개 지정
# min_inpurity_decrease : 결정트리 노드 분리시 불순도 감도 최소량
# max_depth : 결정트리 깊이
# min_samples_split : 결정트리 노드를 나누기 위한 최소 샘플 갯수
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001) ,
          'max_depth' : range(5, 20, 1),
          'min_samples_split' : range(2, 100, 10)}


### 그리드서치 훈련시키기
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),
                 params, cv = splitter, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(cv=StratifiedKFold(n_splits=7, random_state=42, shuffle=True),
             estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [119]:
### 그리드서치를 통해 생성된 훈련모델을 통해
# 평가해보기, 훈련데이터 전체 사용
dt= gs.best_estimator_
print('훈련data = ',dt.score(train_input, train_target))
print('최종data = ',dt.score(test_input,test_target))

훈련data =  0.9230325187608236
최종data =  0.8692307692307693


## 랜덤서치(RandomSerch) <= GridSerch의 개선, 확장 Library

In [124]:
### 랜덤서치 사용시 아래 조건 만족해야 합니다.
# - 하이퍼파라미터값이 수치인 것들만 사용
# - 범위나 간격을 미리 정하기 어려울 경우
# - 넘 많은 매개변수가 있어서 그리드서치 수행시간이 오래 걸리는 경우
# - 매개변수를 샘플링할때 확률분포객체를 전달...

# 사용 모듈 : uniform(실수값), randint(정수값)

In [121]:
from scipy.stats import uniform, randint

In [122]:
### 범위 설정
rgen = randint(0, 10)
rgen

### 범위 내에서 임의값 출력하기(확률적으로 고르게 뽑아냅니다)
print(rgen.rvs(100))

[4 5 9 7 2 2 6 4 5 6 8 7 6 2 2 4 9 5 3 9 1 5 4 9 6 6 2 2 3 6 1 8 7 5 4 0 3
 4 4 1 5 3 4 4 4 6 0 7 3 9 0 0 1 6 1 7 8 6 7 1 5 3 1 8 8 2 0 2 2 8 5 1 5 5
 7 4 3 9 3 1 7 3 0 8 9 4 6 1 7 8 0 4 0 1 6 7 6 4 5 5]


In [123]:
# 실수값
# 범위 설정
ugen = uniform(0,1)

# 임의값 50개 조회
print(ugen.rvs(50))

[0.10531445 0.34890843 0.91019472 0.57880848 0.74427087 0.73195704
 0.92755728 0.35401325 0.28692081 0.07149911 0.65600383 0.07347484
 0.42992442 0.95735985 0.54843494 0.67759041 0.64846499 0.64515144
 0.90825231 0.29565649 0.13393733 0.46104903 0.16003521 0.79035825
 0.28672608 0.44499072 0.55720883 0.12008565 0.83498861 0.45735795
 0.89870691 0.35132392 0.14821096 0.41081708 0.10772839 0.71365283
 0.23915639 0.37311219 0.03185621 0.0368265  0.97327398 0.27348178
 0.57940818 0.74201002 0.07464942 0.30755602 0.10456006 0.06915185
 0.41044332 0.14591325]


In [134]:
### 하이퍼파라미터 찾을 매개변수 정의하기
params = {'min_impurity_decrease' : uniform(0.0001, 0.001), 
         'max_depth' : randint(20,50),
         'min_samples_split' : randint(2,25)}

# 랜덤서치 클래스(모델) : RandomizedSearchCV()
# 사용방법은 그리드서치와 동일
# 결정트리 모델 사용, 훈련반복 횟수 100회, 코어 전체 

### 10 - 폴드 지정하기
splitter = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42)

### 랜덤서치 훈련시키기
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),
                 params, n_iter=100, n_jobs = -1, random_state=42)
rs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000029E1B8997E0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000029E1B8991B0>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000029E1B89B5E0>},
                   random_state=42)

In [135]:
print(rs.best_params_)

{'max_depth': 29, 'min_impurity_decrease': 0.000437615171403628, 'min_samples_split': 16}


In [136]:
print(rs.cv_results_.keys())

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_max_depth', 'param_min_impurity_decrease', 'param_min_samples_split', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])


In [139]:
rs.cv_results_['mean_test_score'].max()

0.8695409787517583

In [138]:
len(rs.cv_results_['mean_test_score'])

100

In [141]:
### 훈련 100회 중에 가장 큰값이 최종 평가점수
print(np.max(rs.cv_results_['mean_test_score']))

0.8695409787517583


In [142]:
dt = rs.best_estimator_
print(dt.score(test_input, test_target))

0.8623076923076923


In [143]:
## 사람이 판단하는 주관적 판단을 최소화할수있다

In [ ]:
## 앙상블 - 결정트리기반으로 합쳐놓은것